In [13]:
import pandas as pd
import pickle 

with open("frames.bin", "rb") as f:
    df = pickle.load(f) 

df = df[df['lon'] != 0]

from sklearn import decomposition; 
from sklearn import datasets; 


df2 = df.copy() 
del df2['valid'] 

pca = decomposition.PCA(n_components=4); 
pca.fit(df2)
df

,lon,lat,sin_time,cos_time,valid
1659,-80.0913,40.4012,0.119415,-0.992844,0
497,-77.4874,39.0438,-0.424660,-0.905353,0
17622,54.8454,54.8454,-0.254813,0.966990,1
3191,-77.4874,39.0438,-0.032792,0.999462,0
238,-122.6760,45.5235,-0.423804,-0.905754,0
...,...,...,...,...,...
3303,-121.8950,37.3394,0.675644,0.737228,0
16660,54.8865,54.8865,-0.318753,0.947838,1
17410,54.2458,54.2458,-0.268429,0.963299,1
16454,52.6720,52.6720,-0.349049,0.937104,1


In [3]:
pca

PCA(n_components=4)

In [5]:
pca.components_

array([[ 0.9973009 ,  0.07307337, -0.00144518,  0.00700819],
       [ 0.07322976, -0.99699656,  0.00123313, -0.02517419],
       [-0.0010886 , -0.01024017,  0.89486506,  0.44621798],
       [-0.0052094 , -0.02352191, -0.44633275,  0.89454271]])

In [6]:
data = pca.components_ 
dataset = pd.DataFrame({'v1': data[:, 0], 'v2': data[:, 1], 'v3':data[:,2], 'v4':data[:,3]})
dataset

,v1,v2,v3,v4
0,0.997301,0.073073,-0.001445,0.007008
1,0.073230,-0.996997,0.001233,-0.025174
2,-0.001089,-0.010240,0.894865,0.446218
3,-0.005209,-0.023522,-0.446333,0.894543


In [9]:
pca_dataset = pd.DataFrame(pca.transform(df2), columns=['PCA%i' % i for i in range(4)], index=df.index)

In [10]:
pca_dataset

,PCA0,PCA1,PCA2,PCA3
1659,-56.214995,1.649941,-0.308308,-0.907770
497,-53.715914,3.191073,-0.745076,-0.568303
17622,79.427259,-2.919293,-0.063480,-0.030278
3191,-53.703131,3.143604,0.455557,0.960732
238,-98.309056,-6.578304,-0.761650,-0.486053
...,...,...,...,...
3303,-98.118278,1.598403,1.038293,0.681382
16660,79.471209,-2.956857,-0.129709,-0.020053
17410,78.785456,-2.365327,-0.070519,-0.010275
16454,77.100834,-0.910943,-0.136522,0.047493


In [14]:
from imblearn.over_sampling import SMOTE, ADASYN
pca_dataset = pd.concat([pca_dataset, df.valid], axis=1)
x = pca_dataset.iloc[:,0:4].to_numpy()
y = pca_dataset.iloc[:,4:5]

sX_resampled, sy_resampled = SMOTE().fit_resample(x, y)
aX_resampled, ay_resampled = ADASYN().fit_resample(x, y)

y = pd.get_dummies(sy_resampled.valid).to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sX_resampled, y, test_size=0.33, random_state=42)

In [18]:
pca_dataset.to_csv('pca.csv')

In [19]:
pca_dataset

,PCA0,PCA1,PCA2,PCA3,valid
1659,-56.214995,1.649941,-0.308308,-0.907770,0
497,-53.715914,3.191073,-0.745076,-0.568303,0
17622,79.427259,-2.919293,-0.063480,-0.030278,1
3191,-53.703131,3.143604,0.455557,0.960732,0
238,-98.309056,-6.578304,-0.761650,-0.486053,0
...,...,...,...,...,...
3303,-98.118278,1.598403,1.038293,0.681382,0
16660,79.471209,-2.956857,-0.129709,-0.020053,1
17410,78.785456,-2.365327,-0.070519,-0.010275,1
16454,77.100834,-0.910943,-0.136522,0.047493,1
